2025-08-23 03:13:10.929556: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755918791.137996      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755918791.198466      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Baixando Alzheimer MRI dataset...
Alzheimer MRI baixado para: /kaggle/input/alzheimer-mri-disease-classification-dataset
Baixando OASIS dataset...
Mounting files to /kaggle/input/oasis-alzheimer-dataset...
OASIS baixado para: /kaggle/input/oasis-alzheimer-dataset
Carregando Alzheimer MRI Parquet...
Parquet carregado com 5120 linhas
Carregando OASIS dataset...
OASIS train: 69148 imagens
OASIS test: 17289 imagens

Distribuição por fonte no df_geral:
source
oasis_train          69148
oasis_test           17289
alzheimer_parquet     5120
Name: count, dtype: int64

Total de imagens: 91557


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Dispositivo: cuda
Contagem por classe: Counter({2: 69788, 3: 15506, 0: 5726, 1: 537})
Pesos normalizados por classe: [0.0825473  0.88019705 0.00677288 0.03048277]


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Época 1/10 - Validação: 100%|██████████| 730/730 [02:30<00:00,  4.86it/s]



    Época 1/10
    ----------------------------
    Treino:
    Loss: 0.4433
    Acurácia: 0.8381
    Precisão: 0.8811
    Recall: 0.8381
    F1 Score: 0.8507
    Validação:
    Loss: 0.2021
    Acurácia: 0.8861
    Precisão: 0.9212
    Recall: 0.8861
    F1 Score: 0.8936
    


Época 2/10 - Validação: 100%|██████████| 730/730 [02:29<00:00,  4.89it/s]



    Época 2/10
    ----------------------------
    Treino:
    Loss: 0.1431
    Acurácia: 0.9485
    Precisão: 0.9541
    Recall: 0.9485
    F1 Score: 0.9500
    Validação:
    Loss: 0.1193
    Acurácia: 0.9624
    Precisão: 0.9640
    Recall: 0.9624
    F1 Score: 0.9629
    


Época 3/10 - Validação: 100%|██████████| 730/730 [02:29<00:00,  4.89it/s]



    Época 3/10
    ----------------------------
    Treino:
    Loss: 0.1087
    Acurácia: 0.9600
    Precisão: 0.9635
    Recall: 0.9600
    F1 Score: 0.9609
    Validação:
    Loss: 0.0826
    Acurácia: 0.9526
    Precisão: 0.9603
    Recall: 0.9526
    F1 Score: 0.9542
    


Época 4/10 - Validação: 100%|██████████| 730/730 [02:29<00:00,  4.89it/s]



    Época 4/10
    ----------------------------
    Treino:
    Loss: 0.0777
    Acurácia: 0.9699
    Precisão: 0.9720
    Recall: 0.9699
    F1 Score: 0.9704
    Validação:
    Loss: 0.0608
    Acurácia: 0.9799
    Precisão: 0.9807
    Recall: 0.9799
    F1 Score: 0.9801
    


Época 5/10 - Validação: 100%|██████████| 730/730 [02:28<00:00,  4.91it/s]



    Época 5/10
    ----------------------------
    Treino:
    Loss: 0.0774
    Acurácia: 0.9715
    Precisão: 0.9734
    Recall: 0.9715
    F1 Score: 0.9720
    Validação:
    Loss: 0.0558
    Acurácia: 0.9838
    Precisão: 0.9843
    Recall: 0.9838
    F1 Score: 0.9838
    


Época 6/10 - Validação: 100%|██████████| 730/730 [02:29<00:00,  4.89it/s]



    Época 6/10
    ----------------------------
    Treino:
    Loss: 0.0566
    Acurácia: 0.9772
    Precisão: 0.9784
    Recall: 0.9772
    F1 Score: 0.9775
    Validação:
    Loss: 0.0826
    Acurácia: 0.9776
    Precisão: 0.9785
    Recall: 0.9776
    F1 Score: 0.9778
    


Época 7/10 - Validação: 100%|██████████| 730/730 [02:29<00:00,  4.89it/s]



    Época 7/10
    ----------------------------
    Treino:
    Loss: 0.0536
    Acurácia: 0.9797
    Precisão: 0.9807
    Recall: 0.9797
    F1 Score: 0.9800
    Validação:
    Loss: 0.0387
    Acurácia: 0.9868
    Precisão: 0.9870
    Recall: 0.9868
    F1 Score: 0.9869
    


Época 8/10 - Validação: 100%|██████████| 730/730 [02:28<00:00,  4.90it/s]



    Época 8/10
    ----------------------------
    Treino:
    Loss: 0.0439
    Acurácia: 0.9825
    Precisão: 0.9832
    Recall: 0.9825
    F1 Score: 0.9827
    Validação:
    Loss: 0.0667
    Acurácia: 0.9683
    Precisão: 0.9727
    Recall: 0.9683
    F1 Score: 0.9692
    


Época 9/10 - Validação: 100%|██████████| 730/730 [02:29<00:00,  4.89it/s]



    Época 9/10
    ----------------------------
    Treino:
    Loss: 0.0445
    Acurácia: 0.9835
    Precisão: 0.9841
    Recall: 0.9835
    F1 Score: 0.9836
    Validação:
    Loss: 0.0330
    Acurácia: 0.9904
    Precisão: 0.9906
    Recall: 0.9904
    F1 Score: 0.9905
    


Época 10/10 - Validação: 100%|██████████| 730/730 [02:29<00:00,  4.88it/s]



    Época 10/10
    ----------------------------
    Treino:
    Loss: 0.0384
    Acurácia: 0.9861
    Precisão: 0.9865
    Recall: 0.9861
    F1 Score: 0.9862
    Validação:
    Loss: 0.0303
    Acurácia: 0.9901
    Precisão: 0.9903
    Recall: 0.9901
    F1 Score: 0.9901
    
Tempo de execução: 26207.65 segundos
Matriz de Confusão - Treino:
[[ 4099     4    11    23]
 [    1   383     1     3]
 [  151    16 49702   552]
 [   43    15   102 11043]]

Matriz de Confusão - Validação:
[[ 724    0    1    5]
 [   0   67    0    1]
 [  14    0 8822   63]
 [  18    0   14 1945]]


Treinamento concluído com sucesso!


In [1]:
import kagglehub
import os
import pandas as pd
from io import BytesIO
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision import transforms
from transformers import ViTForImageClassification, ViTImageProcessor
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time
import numpy as np
from collections import Counter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Mapeia os rótulos numéricos para nomes das classes
label_map = {
    0: 'Mild_Dementia',
    1: 'Moderate_Dementia',
    2: 'Non_Demented',
    3: 'Very_mild_Dementia'
}

# Mapeia os nomes das pastas para rótulos numéricos
class_map = {
    'Mild Dementia': 0,
    'Moderate Dementia': 1,
    'Non Demented': 2,
    'Very mild Dementia': 3
}

# Faz o download dos datasets via kagglehub
print("Baixando Alzheimer MRI dataset...")
path_alzheimer = kagglehub.dataset_download("borhanitrash/alzheimer-mri-disease-classification-dataset")
print(f"Alzheimer MRI baixado para: {path_alzheimer}")

print("Baixando OASIS dataset...")
path_oasis = kagglehub.dataset_download("kushagrasharma133/oasis-alzheimer-dataset")
print(f"OASIS baixado para: {path_oasis}")

# Cria um dataframe a partir das imagens contidas nas pastas
def criar_dataframe_de_imagens(base_path, class_map, source_name):
    dados = []
    for label_nome, label_id in class_map.items():
        pasta = os.path.join(base_path, label_nome)
        if not os.path.exists(pasta):
            print(f"Pasta não encontrada: {pasta}")
            continue
        for nome_arquivo in os.listdir(pasta):
            if nome_arquivo.lower().endswith(('.png', '.jpg', '.jpeg')):
                caminho_completo = os.path.join(pasta, nome_arquivo)
                dados.append({
                    'image_path': caminho_completo,
                    'label': label_id,
                    'source': source_name
                })
    return pd.DataFrame(dados)

# Carrega os dados do arquivo Parquet do Alzheimer MRI
print("Carregando Alzheimer MRI Parquet...")
parquet_files = []
for root, dirs, files in os.walk(path_alzheimer):
    for file in files:
        if file.endswith('.parquet') and 'train' in file:
            parquet_files.append(os.path.join(root, file))

if parquet_files:
    df_parquet = pd.read_parquet(parquet_files[0])
    print(f"Parquet carregado com {len(df_parquet)} linhas")
    
    # Converte os bytes da imagem em objetos BytesIO
    df_parquet['image_path'] = df_parquet['image'].apply(lambda x: BytesIO(x['bytes']))
    df_parquet_final = df_parquet[['image_path', 'label']].copy()
    df_parquet_final['source'] = 'alzheimer_parquet'
else:
    print("Nenhum arquivo Parquet encontrado!")
    df_parquet_final = pd.DataFrame(columns=['image_path', 'label', 'source'])

# Carrega os dados OASIS do caminho correto (usando o path baixado)
print("Carregando OASIS dataset...")
df_train_oasis = criar_dataframe_de_imagens(
    os.path.join(path_oasis, 'train'), class_map, 'oasis_train')
df_test_oasis = criar_dataframe_de_imagens(
    os.path.join(path_oasis, 'test'), class_map, 'oasis_test')

print(f"OASIS train: {len(df_train_oasis)} imagens")
print(f"OASIS test: {len(df_test_oasis)} imagens")

# Junta todos os dataframes
df_geral = pd.concat([df_parquet_final, df_train_oasis, df_test_oasis], ignore_index=True)

# Mostra a distribuição de amostras por fonte
print("\nDistribuição por fonte no df_geral:")
print(df_geral['source'].value_counts())
print(f"\nTotal de imagens: {len(df_geral)}")

# Carregar o processador ViT primeiro
model_name = "google/vit-base-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_name)

# Dataset customizado
class AlzheimerUnifiedDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        entrada = self.df.iloc[idx]['image_path']
        label = self.df.iloc[idx]['label']

        # Se for caminho (string), abra a imagem
        if isinstance(entrada, str):
            image = Image.open(entrada).convert('RGB')
        else:
            # Se for BytesIO (do Parquet)
            entrada.seek(0)  # Volta para o início do buffer
            image = Image.open(entrada).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

# Split em treino, validação e teste
df_trainval, df_test = train_test_split(df_geral, test_size=0.15, stratify=df_geral['label'], random_state=42)
df_train, df_val = train_test_split(df_trainval, test_size=0.15, stratify=df_trainval['label'], random_state=42)

# Transform usando o processador ViT
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=processor.image_mean,
        std=processor.image_std
    )
])

# Carregar os datasets
train_dataset = AlzheimerUnifiedDataset(df_train, transform=transform)
val_dataset = AlzheimerUnifiedDataset(df_val, transform=transform)
test_dataset = AlzheimerUnifiedDataset(df_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=16, num_workers=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo: {device}")

# Calcula pesos das classes
labels_all = df_geral['label'].values
class_counts = Counter(labels_all)
total_samples = sum(class_counts.values())

class_weights = []
for i in range(len(label_map)):
    freq = class_counts[i] / total_samples
    weight = 1.0 / freq if freq > 0 else 1.0
    class_weights.append(weight)

# Normaliza os pesos
class_weights = np.array(class_weights)
class_weights = class_weights / class_weights.sum()

print("Contagem por classe:", class_counts)
print("Pesos normalizados por classe:", class_weights)

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

# Carregar o ViT pré-treinado
model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=4,
    ignore_mismatched_sizes=True
)

model = model.to(device)

# Função de perda com pesos
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Treinamento
num_epochs = 10
train_losses, val_losses = [], []

start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    y_train_true, y_train_pred = [], []

    for images, labels in tqdm(train_loader, desc=f'Época {epoch + 1}/{num_epochs} - Treino'):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        # Forward pass - o ViT espera pixel_values como entrada
        outputs = model(pixel_values=images)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Métricas treino
        _, preds = torch.max(outputs.logits, 1)
        y_train_true.extend(labels.cpu().numpy())
        y_train_pred.extend(preds.cpu().numpy())

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    train_accuracy = accuracy_score(y_train_true, y_train_pred)
    train_precision = precision_score(y_train_true, y_train_pred, average='weighted', zero_division=0)
    train_recall = recall_score(y_train_true, y_train_pred, average='weighted', zero_division=0)
    train_f1 = f1_score(y_train_true, y_train_pred, average='weighted', zero_division=0)

    # Validação
    model.eval()
    val_loss = 0.0
    y_val_true, y_val_pred = [], []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f'Época {epoch+1}/{num_epochs} - Validação'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(pixel_values=images)
            loss = criterion(outputs.logits, labels)
            val_loss += loss.item()

            # Métricas validação
            _, preds = torch.max(outputs.logits, 1)
            y_val_true.extend(labels.cpu().numpy())
            y_val_pred.extend(preds.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    val_accuracy = accuracy_score(y_val_true, y_val_pred)
    val_precision = precision_score(y_val_true, y_val_pred, average='weighted', zero_division=0)
    val_recall = recall_score(y_val_true, y_val_pred, average='weighted', zero_division=0)
    val_f1 = f1_score(y_val_true, y_val_pred, average='weighted', zero_division=0)

    print(f"""
    Época {epoch+1}/{num_epochs}
    ----------------------------
    Treino:
    Loss: {avg_train_loss:.4f}
    Acurácia: {train_accuracy:.4f}
    Precisão: {train_precision:.4f}
    Recall: {train_recall:.4f}
    F1 Score: {train_f1:.4f}
    Validação:
    Loss: {avg_val_loss:.4f}
    Acurácia: {val_accuracy:.4f}
    Precisão: {val_precision:.4f}
    Recall: {val_recall:.4f}
    F1 Score: {val_f1:.4f}
    """)

end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time:.2f} segundos")

# Matriz de confusão
cm_train = confusion_matrix(y_train_true, y_train_pred)
cm_val = confusion_matrix(y_val_true, y_val_pred)

print("Matriz de Confusão - Treino:")
print(cm_train)
print("\nMatriz de Confusão - Validação:")
print(cm_val)

# Plot das matrizes de confusão
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=list(label_map.values()))
disp_train.plot(cmap=plt.cm.Blues, ax=ax1, xticks_rotation=45)
ax1.set_title("Matriz de Confusão - Treino")

disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val, display_labels=list(label_map.values()))
disp_val.plot(cmap=plt.cm.Blues, ax=ax2, xticks_rotation=45)
ax2.set_title("Matriz de Confusão - Validação")

plt.tight_layout()
plt.show()

print("Treinamento concluído com sucesso!")